# Lab 4: Statistical Models of Genetic Inheritance and PwLogLike Class

## Overview

This lab explores the statistical models used in Bonsai v3 for genetic inheritance and relationship inference. We'll examine the mathematical foundations and key algorithms in the `likelihoods.py` module, with particular focus on the `PwLogLike` class. Through practical examples, you'll learn:

1. The probabilistic models of IBD segment length and count distributions
2. How the `PwLogLike` class computes relationship likelihoods
3. How Bonsai integrates genetic and age-based evidence
4. How to use the Bonsai v3 code to infer relationships from IBD data

In [ ]:
# Standard imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inspect
import importlib
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.dirname(os.getcwd()))

# Cross-compatibility setup
from scripts_support.lab_cross_compatibility import setup_environment, is_jupyterlite, save_results, save_plot

# Set up environment-specific paths
DATA_DIR, RESULTS_DIR = setup_environment()

# Set visualization styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook")

In [ ]:
# Setup Bonsai module paths
if not is_jupyterlite():
    # In local environment, add the utils directory to system path
    utils_dir = os.getenv('PROJECT_UTILS_DIR', os.path.join(os.path.dirname(DATA_DIR), 'utils'))
    bonsaitree_dir = os.path.join(utils_dir, 'bonsaitree')
    
    # Add to path if it exists and isn't already there
    if os.path.exists(bonsaitree_dir) and bonsaitree_dir not in sys.path:
        sys.path.append(bonsaitree_dir)
        print(f"Added {bonsaitree_dir} to sys.path")
else:
    # In JupyterLite, use a simplified approach
    print("⚠️ Running in JupyterLite: Some Bonsai functionality may be limited.")
    print("This notebook is primarily designed for local execution where the Bonsai codebase is available.")

In [ ]:
# Helper functions for exploring modules
def display_module_classes(module_name):
    """Display classes and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all classes
        classes = inspect.getmembers(module, inspect.isclass)
        
        # Filter classes defined in this module (not imported)
        classes = [(name, cls) for name, cls in classes if cls.__module__ == module_name]
        
        # Print info for each class
        for name, cls in classes:
            print(f"\n## {name}")
            
            # Get docstring
            doc = inspect.getdoc(cls)
            if doc:
                print(f"Docstring: {doc}")
            else:
                print("No docstring available")
            
            # Get methods
            methods = inspect.getmembers(cls, inspect.isfunction)
            if methods:
                print("\nMethods:")
                for method_name, method in methods:
                    if not method_name.startswith('_'):  # Skip private methods
                        print(f"- {method_name}")
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def display_module_functions(module_name):
    """Display functions and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all functions
        functions = inspect.getmembers(module, inspect.isfunction)
        
        # Filter functions defined in this module (not imported)
        functions = [(name, func) for name, func in functions if func.__module__ == module_name]
        
        # Print info for each function
        for name, func in functions:
            if name.startswith('_'):  # Skip private functions
                continue
                
            print(f"\n## {name}")
            
            # Get signature
            sig = inspect.signature(func)
            print(f"Signature: {name}{sig}")
            
            # Get docstring
            doc = inspect.getdoc(func)
            if doc:
                print(f"Docstring: {doc}")
            else:
                print("No docstring available")
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def view_function_source(module_name, function_name):
    """Display the source code of a function"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Get the function
        func = getattr(module, function_name)
        
        # Get the source code
        source = inspect.getsource(func)
        
        # Print the source code
        from IPython.display import display, Markdown
        display(Markdown(f"```python\n{source}\n```"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except AttributeError:
        print(f"Function {function_name} not found in module {module_name}")
    except Exception as e:
        print(f"Error processing function {function_name}: {e}")

## Exploring the Bonsai v3 Likelihoods Module

Let's begin by exploring the `likelihoods.py` module in Bonsai v3, which implements the statistical models for relationship inference.

In [ ]:
try:
    # Import Bonsai v3 modules
    from utils.bonsaitree.bonsaitree.v3 import likelihoods
    from utils.bonsaitree.bonsaitree.v3 import ibd
    from utils.bonsaitree.bonsaitree.v3 import moments
    
    print("✅ Successfully imported Bonsai v3 modules")
except ImportError as e:
    print(f"❌ Failed to import Bonsai v3 modules: {e}")
    print("This lab requires access to the Bonsai v3 codebase.")

In [ ]:
# Display classes in the likelihoods module
try:
    display_module_classes('utils.bonsaitree.bonsaitree.v3.likelihoods')
except Exception as e:
    print(f"Could not display likelihoods module classes: {e}")
    print("\nThe core class in the likelihoods module is PwLogLike, which computes pairwise relationship likelihoods")
    print("It integrates genetic evidence (IBD) and demographic evidence (age) to infer relationships")

## Statistical Models of Genetic Inheritance

Bonsai v3 uses sophisticated statistical models to describe genetic inheritance patterns. Let's examine these models and their mathematical foundations.

### IBD Segment Length Distributions

IBD segment lengths follow specific distributions based on the relationship between individuals. The key parameter is λ (lambda), which is related to the meiotic distance between individuals.

Let's look at the function that computes this parameter in Bonsai v3:

In [ ]:
try:
    # View the function for computing lambda
    view_function_source('bonsaitree.bonsaitree.v3.moments', 'get_lam_a_m')
except Exception as e:
    print(f"Could not display the get_lam_a_m function: {e}")
    print("\nThe get_lam_a_m function computes the lambda parameter for the IBD segment length distribution.")
    print("It takes relationship parameters (a, m) and returns the expected rate parameter for the exponential distribution.")
    print("The parameter a represents the number of meioses to the MRCA through one lineage,")
    print("and m represents the number of meioses to the MRCA through the other lineage.")

Let's visualize IBD segment length distributions for different relationships:

In [ ]:
def plot_segment_length_distributions():
    """Plot IBD segment length distributions for different relationships"""
    # Define relationships to plot
    relationships = [
        ("Parent-Child", 0, 1),
        ("Full Siblings", 1, 1),
        ("Grandparent-Grandchild", 0, 2),
        ("First Cousins", 2, 2),
        ("Second Cousins", 3, 3),
    ]
    
    plt.figure(figsize=(12, 8))
    
    # Generate data for each relationship
    cM_values = np.linspace(0, 100, 1000)
    for rel_name, a, m in relationships:
        try:
            # Use the actual Bonsai function if available
            lambda_val = moments.get_lam_a_m(a, m)
        except:
            # Fall back to a simplified computation if Bonsai is not available
            lambda_val = 100 / (a + m + 0.5)  # Very approximate
        
        # Generate exponential distribution with this lambda
        pdf_values = lambda_val * np.exp(-lambda_val * cM_values)
        
        # Plot the distribution
        plt.plot(cM_values, pdf_values, label=f"{rel_name} (a={a}, m={m})")
    
    plt.xlabel('Segment Length (cM)')
    plt.ylabel('Probability Density')
    plt.title('IBD Segment Length Distributions by Relationship')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.xlim(0, 100)
    plt.tight_layout()
    plt.show()

# Plot the distributions
plot_segment_length_distributions()

### IBD Segment Count Distributions

The expected number of IBD segments (η, eta) also varies by relationship. Let's look at the function that computes this parameter:

In [ ]:
try:
    # View the function for computing eta
    view_function_source('bonsaitree.bonsaitree.v3.moments', 'get_eta')
except Exception as e:
    print(f"Could not display the get_eta function: {e}")
    print("\nThe get_eta function computes the expected number of IBD segments for a relationship.")
    print("It takes relationship parameters, length threshold, and other parameters to determine the expected count.")

Let's visualize the expected segment count for different relationships:

In [ ]:
def plot_segment_count_expectations():
    """Plot expected IBD segment counts for different relationships"""
    # Define relationships to examine
    relationships = [
        ("Parent-Child", 0, 1),
        ("Full Siblings", 1, 1),
        ("Grandparent-Grandchild", 0, 2),
        ("First Cousins", 2, 2),
        ("Second Cousins", 3, 3),
        ("Third Cousins", 4, 4),
    ]
    
    # Define segment length thresholds to examine
    thresholds = [1, 2, 5, 10, 15]
    
    # Create a DataFrame to store the results
    data = []
    
    for rel_name, a, m in relationships:
        for threshold in thresholds:
            try:
                # Use the actual Bonsai function if available
                eta = moments.get_eta(a, m, threshold)
            except:
                # Fall back to a simplified computation if Bonsai is not available
                c = 3500  # Approximate genome length in cM
                lambda_val = 100 / (a + m + 0.5)  # Very approximate
                eta = c / (a + m) * np.exp(-lambda_val * threshold)
            
            data.append({
                'relationship': rel_name,
                'threshold': threshold,
                'expected_segments': eta
            })
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Create a grouped bar chart
    plt.figure(figsize=(14, 8))
    
    rel_order = [rel for rel, _, _ in relationships]
    
    # Create grouped bar chart
    ax = sns.barplot(data=df, x='relationship', y='expected_segments', hue='threshold',
                    order=rel_order, palette='viridis')
    
    plt.xlabel('Relationship')
    plt.ylabel('Expected Number of Segments')
    plt.title('Expected IBD Segment Counts by Relationship and Length Threshold')
    plt.legend(title='Minimum Length (cM)')
    plt.grid(axis='y', alpha=0.3)
    plt.yscale('log')  # Log scale for better visualization
    
    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    return df

# Plot the expected segment counts
segment_count_df = plot_segment_count_expectations()

## The PwLogLike Class

The heart of Bonsai v3's relationship inference is the `PwLogLike` class, which computes relationship likelihoods based on IBD data and age information. Let's examine this class in detail.

In [ ]:
try:
    # Look at the PwLogLike class initialization
    view_function_source('bonsaitree.bonsaitree.v3.likelihoods', 'PwLogLike.__init__')
except Exception as e:
    print(f"Could not display the PwLogLike.__init__ method: {e}")
    print("\nThe PwLogLike class initializes with:")
    print("1. bio_info: Dictionary with age, sex, and coverage information")
    print("2. unphased_ibd_seg_list: List of unphased IBD segments")
    print("3. phased_ibd_seg_list: Optional list of phased IBD segments")
    print("4. condition_pair_set: Set of individual pairs to condition on")
    print("5. Other parameters for background IBD and likelihood computation")

Let's look at the key methods for likelihood computation:

In [ ]:
try:
    # Display methods for genetic likelihood computation
    view_function_source('bonsaitree.bonsaitree.v3.likelihoods', 'PwLogLike.get_pw_gen_ll')
except Exception as e:
    print(f"Could not display the get_pw_gen_ll method: {e}")
    print("\nThe get_pw_gen_ll method computes the genetic component of the likelihood.")
    print("It assesses how well the observed IBD patterns match the expected patterns for various relationships.")

In [ ]:
try:
    # Display methods for age-based likelihood computation
    view_function_source('bonsaitree.bonsaitree.v3.likelihoods', 'PwLogLike.get_pw_age_ll')
except Exception as e:
    print(f"Could not display the get_pw_age_ll method: {e}")
    print("\nThe get_pw_age_ll method computes the age-based component of the likelihood.")
    print("It evaluates how well the observed age differences match the expected distributions for various relationships.")

In [ ]:
try:
    # Display methods for combined likelihood computation
    view_function_source('bonsaitree.bonsaitree.v3.likelihoods', 'PwLogLike.get_pw_ll')
except Exception as e:
    print(f"Could not display the get_pw_ll method: {e}")
    print("\nThe get_pw_ll method combines genetic and age-based likelihoods.")
    print("It weights and sums the components to produce a final likelihood score for each relationship.")

## Using PwLogLike with Real Data

Now let's see how to use the `PwLogLike` class with real data to infer relationships. We'll use sample data from the class_data directory.

In [ ]:
# First, let's load some IBD data from the class_data directory
def load_ibd_data():
    """Load IBD data from the class_data directory"""
    try:
        # Check for IBD files in class_data
        ibd_file_path = os.path.join(DATA_DIR, "ped_sim_run2.seg")
        dict_file_path = os.path.join(DATA_DIR, "ped_sim_run2.seg_dict.txt")
        fam_file_path = os.path.join(DATA_DIR, "ped_sim_run2-everyone.fam")
        
        # Check if files exist
        files_exist = all(os.path.exists(f) for f in [ibd_file_path, dict_file_path, fam_file_path])
        
        if files_exist:
            # Read the IBD segments
            ibd_df = pd.read_csv(ibd_file_path, sep="\t")
            
            # Read the ID mapping dictionary
            dict_df = pd.read_csv(dict_file_path, sep="\t", header=None, names=["original_id", "mapped_id"])
            id_mapping = dict(zip(dict_df["mapped_id"], dict_df["original_id"]))
            
            # Read the FAM file for demographic info
            fam_df = pd.read_csv(fam_file_path, sep="\s+", header=None, 
                              names=["fam_id", "indiv_id", "father_id", "mother_id", "sex", "phenotype"])
            
            print(f"Loaded {len(ibd_df)} IBD segments")
            print(f"Loaded information for {len(fam_df)} individuals")
            
            return ibd_df, id_mapping, fam_df
        else:
            print("IBD data files not found in class_data directory")
            return None, None, None
    except Exception as e:
        print(f"Error loading IBD data: {e}")
        return None, None, None

# Load the IBD data
ibd_df, id_mapping, fam_df = load_ibd_data()

In [ ]:
# Convert the IBD data to Bonsai's format
def prepare_bonsai_inputs(ibd_df, id_mapping, fam_df):
    """Prepare inputs for the PwLogLike class"""
    if ibd_df is None or id_mapping is None or fam_df is None:
        print("Data loading failed, cannot prepare inputs")
        return None, None
    
    # Convert IBD data to unphased format
    unphased_segments = []
    for _, row in ibd_df.iterrows():
        # Get mapped IDs
        id1 = str(row['ID1'])
        id2 = str(row['ID2'])
        
        # Get segment information
        chromosome = int(row['Chr'])
        start_bp = int(row['StartBP'])
        end_bp = int(row['EndBP'])
        seg_cm = float(row['Length(cM)'])
        is_full_ibd = 1 if row['IBDType'] == 2 else 0  # 2 = IBD2, 1 = IBD1
        
        # Create unphased segment
        unphased_segments.append([id1, id2, chromosome, start_bp, end_bp, is_full_ibd, seg_cm])
    
    # Prepare bio_info dictionary
    bio_info = {}
    for _, row in fam_df.iterrows():
        indiv_id = str(row['indiv_id'])
        
        # Add sex information
        sex = int(row['sex'])
        bio_info.setdefault(indiv_id, {})['sex'] = sex
        
        # Add dummy age and coverage information
        # (In a real application, you would use actual age information if available)
        bio_info[indiv_id]['age'] = 30 + int(indiv_id) % 50  # Dummy age
        bio_info[indiv_id]['cov'] = 1.0  # Assume perfect coverage
    
    return unphased_segments, bio_info

# Prepare inputs for Bonsai
unphased_segments, bio_info = prepare_bonsai_inputs(ibd_df, id_mapping, fam_df)

In [ ]:
# Display a sample of the prepared data
print("Sample of unphased IBD segments:")
for segment in unphased_segments[:5]:
    print(segment)

print("\nSample of bio_info dictionary:")
for person_id, info in list(bio_info.items())[:5]:
    print(f"{person_id}: {info}")

In [ ]:
# Create a PwLogLike instance
try:
    # Create the instance with actual Bonsai code
    pw_ll = likelihoods.PwLogLike(
        bio_info=bio_info,
        unphased_ibd_seg_list=unphased_segments,
        condition_pair_set=None,
        mean_bgd_num=0.1,  # Background IBD segment count
        mean_bgd_len=5.0,  # Background IBD segment length
    )
    
    print("✅ Successfully created PwLogLike instance")
    
    # Display some information about the instance
    print(f"Number of individual pairs with IBD: {len(pw_ll.ibd_stat_dict)}")
    print(f"Number of individuals with biological info: {len(pw_ll.sex_dict)}")
except Exception as e:
    print(f"❌ Could not create PwLogLike instance: {e}")
    print("Will demonstrate with simulated approaches instead")

Now let's compute relationship likelihoods for some pairs of individuals:

In [ ]:
def compute_relationship_likelihoods(pw_ll, pairs_to_analyze=10):
    """Compute relationship likelihoods for individual pairs"""
    # Get pairs with IBD
    pairs = list(pw_ll.ibd_stat_dict.keys())
    
    # Limit to a reasonable number for demonstration
    pairs = pairs[:pairs_to_analyze]
    
    # Define relationship tuples to test
    relationship_tuples = [
        (0, 0, 1, 0, 1),  # Parent-Child
        (1, 0, 1, 0, 0),  # Full Siblings
        (1, 0, 1, 0, 1),  # Half Siblings
        (0, 0, 2, 0, 1),  # Grandparent-Grandchild
        (1, 0, 2, 0, 1),  # Uncle/Aunt-Niece/Nephew
        (2, 0, 2, 0, 0),  # First Cousins
        (2, 0, 2, 0, 1),  # Half First Cousins
        (3, 0, 3, 0, 0),  # Second Cousins
    ]
    
    # Define relationship names for display
    relationship_names = [
        "Parent-Child",
        "Full Siblings",
        "Half Siblings",
        "Grandparent-Grandchild",
        "Uncle/Aunt-Niece/Nephew",
        "First Cousins",
        "Half First Cousins",
        "Second Cousins",
    ]
    
    # Compute likelihoods for each pair
    results = []
    for pair in pairs:
        id1, id2 = pair
        
        # Get IBD statistics
        stats = pw_ll.ibd_stat_dict[pair]
        
        # Compute likelihoods for different relationships
        likelihoods = {}
        for rel_tuple, rel_name in zip(relationship_tuples, relationship_names):
            try:
                # Compute genetic likelihood
                gen_ll = pw_ll.get_pw_gen_ll(id1, id2, rel_tuple)
                
                # Compute age likelihood if available
                if id1 in pw_ll.age_dict and id2 in pw_ll.age_dict:
                    age_ll = pw_ll.get_pw_age_ll(id1, id2, rel_tuple)
                else:
                    age_ll = 0.0
                
                # Combine likelihoods
                combined_ll = pw_ll.get_pw_ll(id1, id2, rel_tuple)
                
                likelihoods[rel_name] = {
                    'genetic_ll': gen_ll,
                    'age_ll': age_ll,
                    'combined_ll': combined_ll
                }
            except Exception as e:
                print(f"Error computing likelihood for {id1}-{id2}, {rel_name}: {e}")
                likelihoods[rel_name] = {
                    'genetic_ll': float('-inf'),
                    'age_ll': float('-inf'),
                    'combined_ll': float('-inf')
                }
        
        # Identify the most likely relationship
        best_rel = max(likelihoods.items(), key=lambda x: x[1]['combined_ll'])[0]
        best_ll = likelihoods[best_rel]['combined_ll']
        
        # Store the results
        results.append({
            'id1': id1,
            'id2': id2,
            'total_half': stats['total_half'],
            'total_full': stats['total_full'],
            'num_half': stats['num_half'],
            'num_full': stats['num_full'],
            'max_seg_cm': stats['max_seg_cm'],
            'best_relationship': best_rel,
            'best_likelihood': best_ll,
            'likelihoods': likelihoods
        })
    
    return results

# Compute likelihoods
try:
    relationship_results = compute_relationship_likelihoods(pw_ll, pairs_to_analyze=10)
    print(f"Computed relationship likelihoods for {len(relationship_results)} pairs")
except Exception as e:
    print(f"Could not compute relationship likelihoods: {e}")
    relationship_results = []

In [ ]:
# Display the relationship inference results
if relationship_results:
    # Create a simplified DataFrame for display
    display_data = []
    for result in relationship_results:
        # Calculate total IBD
        total_ibd = result['total_half'] + 2 * result['total_full']
        
        # Get top 3 relationships
        rel_rankings = sorted(result['likelihoods'].items(), 
                             key=lambda x: x[1]['combined_ll'], 
                             reverse=True)[:3]
        
        top_rels = []
        top_lls = []
        for rel_name, ll_dict in rel_rankings:
            top_rels.append(rel_name)
            top_lls.append(ll_dict['combined_ll'])
        
        # Create a row for the display table
        display_data.append({
            'Pair': f"{result['id1']}-{result['id2']}",
            'Total IBD (cM)': round(total_ibd, 1),
            'Segments': result['num_half'] + result['num_full'],
            'Max Segment (cM)': round(result['max_seg_cm'], 1),
            'Best Relationship': result['best_relationship'],
            'Top 3 Relationships': ', '.join(top_rels),
            'Top 3 Log-Likelihoods': ', '.join([f"{ll:.1f}" for ll in top_lls])
        })
    
    # Create DataFrame and display
    display_df = pd.DataFrame(display_data)
    display(display_df)
else:
    print("No relationship results to display")

## Visualizing Relationship Inference

Let's create some visualizations to better understand the relationship inference process:

In [ ]:
def visualize_likelihood_components(pair_idx=0):
    """Visualize the genetic and age components of relationship likelihoods"""
    if not relationship_results:
        print("No relationship results available")
        return
    
    # Get the result for the specified pair
    result = relationship_results[pair_idx]
    pair_label = f"{result['id1']}-{result['id2']}"
    
    # Extract likelihood components
    rel_names = []
    genetic_lls = []
    age_lls = []
    combined_lls = []
    
    for rel_name, ll_dict in result['likelihoods'].items():
        rel_names.append(rel_name)
        genetic_lls.append(ll_dict['genetic_ll'])
        age_lls.append(ll_dict['age_ll'])
        combined_lls.append(ll_dict['combined_ll'])
    
    # Create a DataFrame for plotting
    plot_df = pd.DataFrame({
        'Relationship': rel_names,
        'Genetic': genetic_lls,
        'Age': age_lls,
        'Combined': combined_lls
    })
    
    # Sort by combined likelihood
    plot_df = plot_df.sort_values('Combined', ascending=False)
    
    # Plot the components
    plt.figure(figsize=(12, 8))
    
    # Create a bar chart
    bar_width = 0.3
    x = np.arange(len(plot_df))
    
    plt.bar(x - bar_width, plot_df['Genetic'], bar_width, label='Genetic')
    plt.bar(x, plot_df['Age'], bar_width, label='Age')
    plt.bar(x + bar_width, plot_df['Combined'], bar_width, label='Combined')
    
    plt.xticks(x, plot_df['Relationship'], rotation=45, ha='right')
    plt.xlabel('Relationship')
    plt.ylabel('Log-Likelihood')
    plt.title(f'Relationship Likelihood Components for Pair {pair_label}')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Display IBD statistics for context
    print(f"IBD Statistics for Pair {pair_label}:")
    print(f"Total IBD1: {result['total_half']:.1f} cM")
    print(f"Total IBD2: {result['total_full']:.1f} cM")
    print(f"Total IBD: {result['total_half'] + 2*result['total_full']:.1f} cM")
    print(f"Number of IBD1 segments: {result['num_half']}")
    print(f"Number of IBD2 segments: {result['num_full']}")
    print(f"Maximum segment length: {result['max_seg_cm']:.1f} cM")

# Visualize likelihood components for the first pair
if relationship_results:
    visualize_likelihood_components(pair_idx=0)
else:
    print("No relationship results available for visualization")

## Summary

In this lab, we've explored the statistical models of genetic inheritance and the relationship inference capabilities of Bonsai v3:

1. **IBD Segment Length Distributions**: We examined how segment lengths follow exponential distributions with parameters related to the meiotic distance between individuals.

2. **IBD Segment Count Distributions**: We learned how the expected number of segments varies by relationship and segment length threshold.

3. **The PwLogLike Class**: We explored Bonsai v3's powerful class for computing relationship likelihoods, which integrates genetic and age-based evidence.

4. **Relationship Inference in Practice**: We demonstrated how to use the actual Bonsai v3 code with real data to infer relationships between individuals.

These concepts are fundamental to Bonsai v3's approach to pedigree reconstruction, which builds on pairwise relationship inferences to construct complete family trees.